In [122]:
import os 
import tensorflow as tf
import pandas as pd
from sklearn import model_selection

import functions

ModuleNotFoundError: ignored

In [120]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [36]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vco4/AAC3-C0xn8DrT1PZLAQbVTbJ20UtFecTjnjLybCRWM-QgdDQqsDUTs0de} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

gpg: keybox '/tmp/tmpu8pew89n/pubring.gpg' created
gpg: /tmp/tmpu8pew89n/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1


··········


In [40]:
!mkdir -p drive
!google-drive-ocamlfuse drive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [0]:
os.chdir("/content/drive/Hamming")

In [125]:
!pwd

/content/drive/Hamming


In [124]:
!ls

drive  hamming-codes.ipynb  h.txt    README.md	visualisation.ipynb
g.txt  hamming.txt	    loss.py  src.py


In [127]:
!git checkout origin/master
!git merge master
!git pull

error: The following untracked working tree files would be overwritten by checkout:
	hamming-codes.ipynb
Please move or remove them before you switch branches.
Aborting
Already up-to-date.
Updating a6fd94c..59dbefd
error: The following untracked working tree files would be overwritten by merge:
	hamming-codes.ipynb
Please move or remove them before you merge.
Aborting


In [0]:
COLUMN_NAMES = ['id', 'plainword', 'codeword', 
                'id_error', 'bin_error', 'defective_codeword']
TRAIN_PATH = "/content/drive/Hamming/hamming.txt"

In [0]:
data = pd.read_csv(TRAIN_PATH, sep=';', names = COLUMN_NAMES)

In [111]:
data.head()

,id,plainword,codeword,id_error,bin_error,defective_codeword
0,0,01101110111110010000111110,0100101101110111110010000111110,9,0000000001000000000000000000000,0100101100110111110010000111110
1,1,00110010000100100100011001,0001100110010000100100100011001,26,0000000000000000000000000010000,0001100110010000100100100001001
2,2,10011010110010011011100100,1011010011010110010011011100100,10,0000000000100000000000000000000,1011010011110110010011011100100
3,3,10001111100101011001110101,1100010001111100101011001110101,21,0000000000000000000001000000000,1100010001111100101010001110101
4,4,01101011110100100000001110,1010001101011110100100000001110,8,0000000010000000000000000000000,1010001111011110100100000001110


In [0]:
def make_features():
  data['dec_defective_codeword'] = data['defective_codeword'][:].apply(lambda x: int(x, 2))

  for j in range(len(data['defective_codeword'][0])):
    data['bin_' + str(j)] = data['defective_codeword'][:].apply(lambda x: int(x[j]))
  
  data['dec_plainword'] = data['plainword'][:].apply(lambda x: int(x, 2))

In [0]:
make_features()

In [114]:
data.head()

,id,plainword,codeword,id_error,bin_error,defective_codeword,dec_defective_codeword,bin_0,bin_1,bin_2,...,bin_22,bin_23,bin_24,bin_25,bin_26,bin_27,bin_28,bin_29,bin_30,dec_plainword
0,0,01101110111110010000111110,0100101101110111110010000111110,9,0000000001000000000000000000000,0100101100110111110010000111110,630973502,0,1,0,...,0,0,0,1,1,1,1,1,0,29090878
1,1,00110010000100100100011001,0001100110010000100100100011001,26,0000000000000000000000000010000,0001100110010000100100100001001,214452489,0,0,0,...,1,0,0,0,0,1,0,0,1,13125913
2,2,10011010110010011011100100,1011010011010110010011011100100,10,0000000000100000000000000000000,1011010011110110010011011100100,1518020324,1,0,1,...,0,1,1,1,0,0,1,0,0,40576740
3,3,10001111100101011001110101,1100010001111100101011001110101,21,0000000000000000000001000000000,1100010001111100101010001110101,1648252021,1,1,0,...,0,0,1,1,1,0,1,0,1,37639797
4,4,01101011110100100000001110,1010001101011110100100000001110,8,0000000010000000000000000000000,1010001111011110100100000001110,1374636046,1,0,1,...,0,0,0,0,0,1,1,1,0,28264462


In [0]:
def split_data():
  train_data, test_data, train_labels, test_labels = \
      model_selection.train_test_split(data.loc[:, 'dec_defective_codeword':'bin_30'], 
                                       data['id_error'], 
                                       test_size = 0.3)

In [0]:
split_data()

In [0]:
# Create feature columns for all features.
feature_columns = []
for key in train_data.keys():
    feature_columns.append(tf.feature_column.numeric_column(key=key))

In [117]:
classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[100, 100, 100],
    n_classes=data.shape[0])

INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpnt5otl_0', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4d5a265860>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpnt5otl_0', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4d5a265860>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [0]:
loss = hamming_distance

In [118]:
classifier.train(
    input_fn=lambda:train_input_fn(train_data, train_labels, 100),
    steps=100
  )

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Create CheckpointSaverHook.


...

INFO:tensorflow:loss = 1198670600.0, step = 1


INFO:tensorflow:Saving checkpoints for 100 into /tmp/tmpnt5otl_0/model.ckpt.


INFO:tensorflow:Saving checkpoints for 100 into /tmp/tmpnt5otl_0/model.ckpt.


INFO:tensorflow:Loss for final step: 1024.3114.


INFO:tensorflow:Loss for final step: 1024.3114.
